In [1]:
import pandas as pd
import numpy as np
# 랜덤포레스트, 그리드서치, 교차검증
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_validate
# 시각화
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
# helpful character encoding module
import chardet
import warnings
warnings.filterwarnings('ignore')

In [2]:
#  ascii 값 확인
# with open('.\SUV_Purchase.csv', 'rb') as rawdata:
#     result = chardet.detect(rawdata.read(10000))
# result

In [3]:
df = pd.read_csv('.\SUV_Purchase.csv', encoding='utf-8')
df

FileNotFoundError: ignored

In [ ]:
# df.to_csv('.\SUV_Purchase.csv', encoding='utf-8')

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
#중복된 행의 데이터만 표시하기
display(df[df.duplicated()])

In [ ]:
# 타겟 변수 0, 1 갯수 확인
df['Purchased'].value_counts()

In [ ]:
# UserID 삭제
df = df.drop(['User ID'], axis=1)

In [ ]:
# 더미변수화 (원하는 컬럼만)
df2 = pd.get_dummies(df, columns = ['Gender'])
df2

In [ ]:
# 수치형, 범주형 변수 나누기
numerical_feats = df2.dtypes[df2.dtypes != "object"].index 
print("Number of Numerical features: ", len(numerical_feats))
categorical_feats = df2.dtypes[df2.dtypes == "object"].index
print("Number of Categorical features: ", len(categorical_feats))

In [ ]:
# 각 칼럼별 데이터 정규 분포 확인(왜도, 첨도)
# 왜도(비대칭도, Skewness)란, 확률변수의 확률분포가 비대칭성을 가진 걸 뜻함, a=0이면 정규분포, 
# a<0이면 오른쪽으로 치우침, a>0이면 왼쪽으로 치우침을 의미
# 첨도(Kurtosis)란, 확률분포의 뾰족한 정도, 
# a=3이면 정규분포, a<3이면 정규분포보다 완만함, a>3이면 정규분포 보다 뾰족함을 의미

for col in numerical_feats: 
    print('{:15}'.format(col), 'Skewness: {:05.2f}'.format(df2[col].skew()) , 
          ' ' , 'Kurtosis: {:06.2f}'.format(df2[col].kurt()) 
         )

## 데이터 시각화

In [ ]:
# 데이터프레임 열을 두 개씩 짝 지을 수 있는 모든 경우의 수에 대해 두 변수 간의 산점도 그리기
# 자기 자신은 히스토그램
grid_df = sns.pairplot(df2)
plt.show()
plt.close()

In [ ]:
# 타겟 변수 분포 확인
# Log를 취해도 똑같음
f, ax = plt.subplots(figsize = (10, 6))
sns.distplot(df2['Purchased'])

In [ ]:
# \변수에 Log를 취해보기 
# df2['Purchased'] = np.log1p(df2['Purchased'])
# f, ax = plt.subplots(figsize = (10, 6))
# sns.distplot(df2['Purchased']) 
# print("Skewness: {:.3f}".format(df2['Purchased'].skew())) 
# print("Kurtosis: {:.3f}".format(df2['Purchased'].kurt()))

In [ ]:
# age변수 분포 확인
f, ax = plt.subplots(figsize = (10, 6))
sns.distplot(df2['Age'])#정규분포

## 훈련/검증/테스트 데이터 분류

In [ ]:
# 데이터셋 구분 및 배열로 변환
data = df2[['Age','EstimatedSalary','Gender_Female','Gender_Male']].to_numpy()#독립변수
target= df2['Purchased'].to_numpy()#종속 변수

In [ ]:
# 훈련, 테스트 세트 나누기 test_size default = 25%
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(
data, target, test_size=0.2, random_state=42)

In [ ]:
train_input.shape, test_input.shape

In [ ]:
# 검증 세트 만들기
sub_input, val_input, sub_target, val_target = train_test_split(
train_input, train_target, test_size=0.2, random_state=42
)
#훈련세트를 또다시 검증세트 20% 떼어내고 나머지를 훈련세트로 만들기

In [ ]:
# 크기 확인
sub_input.shape, val_input.shape
# 훈련세트 5197개 중에서 4157개와 1040개로 나눔

## 결정트리(교차검증, 하이퍼 파라미터 튜닝x)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)

In [ ]:
dt.score(sub_input, sub_target)

In [ ]:
dt.score(val_input, val_target)

In [ ]:
# 결정트리 시각화
import matplotlib.pyplot as plt
from sklearn.tree import plot_tree
plt.figure(figsize=(10, 7))
plot_tree(dt, max_depth=1, filled=True, feature_names=['Age','EstimatedSalary','Gender_Female','Gender_Male'])
plt.show()

## 교차검증만 했을 때

In [ ]:
# 교차검증 함수 사용 
# 훈련 세트 전체를 검증 함수에 전달
scores = cross_validate(dt, train_input, train_target)

In [ ]:
# 교차 검증의 최종 점수는 test_score 키에 담긴 5개의 점수 평균하여 얻음
np.mean(scores['test_score'])

In [ ]:
# 교차검증만 실행 시 왜 점수가 더 낮게 나오는 걸까?

## 결정트리 (교차검증, 그리드서치o)

In [ ]:
# 매개변수 설정 
params = {'min_impurity_decrease': np.arange(0.0001, 0.001, 0.0001), 
         'max_depth': range(5, 20, 1),
         'min_samples_split': range(2, 100, 10)}

In [ ]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=1)
gs.fit(train_input, train_target)

# n_jobs=1로 했더니 UnicodeEncodeError: 'ascii' codec can't encode characters in position 18-20: ordinal not in range(128) 오류 해결
# 병렬처리 문제, dask는 pandas와 유사하지만 안정화 x, dask삭제
# msg = '{0}:{1}:{2}\n'.format(cmd, name, rtype).encode('utf-8')

In [ ]:
print('최적 하이퍼 파라미터: ', gs.best_params_)

In [ ]:
# 최상의 교차검증 점수 확인
np.max(gs.cv_results_['mean_test_score'])

## 랜덤포레스트(교차검증, 그리드서치 o)

In [ ]:
# 그리드서치
params = { 'n_estimators' : [10,100],
            'max_depth':[6,8,10,12],
            'min_samples_leaf':[8,12,18],
            'min_samples_split':[8,16,20]
                    }

In [ ]:
rf = RandomForestClassifier(random_state=0, n_jobs=1)

In [ ]:
# RandomForestClassifier 객체 생성 후 GridSearchCV 수행
grid_cv = GridSearchCV(rf, param_grid=params, cv=5, n_jobs=1)

In [ ]:
grid_cv.fit(train_input, train_target)

In [ ]:
print('최적 하이퍼 파라미터: ', grid_cv.best_params_)
print('최고 예측 정확도: {:.4f}'.format(grid_cv.best_score_))

In [ ]:
#위의 결과로 나온 최적 하이퍼 파라미터로 다시 모델을 학습하여 테스트 세트 데이터에서 예측 성능을 측정
rf_clf1 = RandomForestClassifier(n_estimators = 10, 
                                max_depth = 6,
                                min_samples_leaf = 8,
                                min_samples_split = 30,
                                random_state = 0,
                                n_jobs = 1)

In [ ]:
from sklearn.metrics import accuracy_score
rf_clf1.fit(train_input, train_target)
rfc_pred = rf_clf1.predict(test_input)
print('예측 정확도: {:.4f}'.format(accuracy_score(test_target,rfc_pred)))

## F점수

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(test_target, rfc_pred, target_names=['구매결정x(0)', '구매결정o(1)']))

In [ ]:
# 특성 중요도 평가
ftr_importances_values = rf_clf1.feature_importances_
ftr_importances = pd.Series(ftr_importances_values, index = df2[['Age','EstimatedSalary','Gender_Female','Gender_Male']].columns)
ftr_top = ftr_importances.sort_values(ascending=False)[:20]

plt.figure(figsize=(8,6))
plt.title('Feature Importances')
sns.barplot(x=ftr_top, y=ftr_top.index)
plt.show()

## 교차검증, 그리드서치x

In [ ]:
rfc = RandomForestClassifier(n_estimators=10)
rfc

In [ ]:
# rfc.fin()에 훈련 데이터를 입력해 Random Forest 모듈을 학습
rfc.fit(train_input, train_target)
#test data를 입력해 target data를 예측 (매번 달라짐)
prediction = rfc.predict(test_input)
#예측 결과 precision과 실제 test data의 target을 비교
print(prediction==test_target)

In [ ]:
#test data 정확도 측정
rfc.score(test_input, test_target)

In [ ]:
# test data 분류성능 평가
print(classification_report(prediction, test_target))

## XGboost(교차검증, 그리드서치 o)

In [ ]:
from xgboost import plot_importance
from xgboost import XGBClassifier

In [ ]:
# !pip install xgboost

In [ ]:
# 객체 생성, 일단 트리 100개 만듬
xgb = XGBClassifier(n_estimators=100)

In [ ]:
# 후보 파라미터 선정
params = {'max_depth':range(3, 10, 1), 'learning_rate':np.arange(0.01,0.2,0.05), 'min_child_weight':range(1,3,1), 'colsample_bytree':np.arange(0.5,1,0.1)}
# 'min_child_weight' child 에서 필요한 모든 관측치에 대한 가중치의 최소 합, 
# 이 값보다 샘플 수가 작으면 leaf node가 되는 것, 너무 크면 under-fitting 될 수 있음
#  colsample_bytree 각 트리마다 feature 샘플링 비율 일반적으로 0.5 ~ 1

In [ ]:
# gridsearchcv 객체 정보 입력(어떤 모델, 파라미터 후보, 교차검증 5번)
gs = GridSearchCV(xgb, params, cv=5)

In [ ]:
# 파라미터 튜닝 시작
gs.fit(train_input, train_target, early_stopping_rounds=30)
# early_stopping_rounds 최대한 몇 개의 트리를 완성해볼 것인지 valid loss에 더이상 진전이 없으면 멈춤, 
# 과적합을 방지할 수 있음, n_estimators 가 높을때 주로 사용.

In [ ]:
#튜닝된 최상의 매개변수 출력
print('최적 하이퍼 파라미터: ',gs.best_params_)

In [ ]:
# 1차적으로 튜닝된 파라미터를 가지고 객체 생성
xgb_model = XGBClassifier(n_estimators=1000, learning_rate=0.01, max_depth=3, min_child_weight=1, colsample_bytree=0.5, reg_alpha=0.03)

In [ ]:
# 학습
model = xgb_model.fit(train_input, train_target)

In [ ]:
# 테스트 성능 평가
xgb_pred2 = model.predict(test_input)

## 이진분류 성능평가(그리드서치o)

## F점수

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(test_target, xgb_pred2, target_names=['구매결정x(0)', '구매결정o(1)']))

## ROC Curve

In [ ]:
buildROC(test_target,xgb_pred2)

## 이진분류 성능평가(그리드서치x)

### F점수

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(test_target, xgb_pred, target_names=['구매결정x(0)', '구매결정o(1)']))

In [ ]:
# macro: 단순평균
# weighted: 각 클래스에 속하는 표본의 갯수로 가중평균
# accuracy: 정확도. 전체 학습데이터의 개수에서 각 클래스에서 자신의 클래스를 정확하게 맞춘 개수의 비율.

# 해석
# 구매결정x라고 예측한 데이터의 92%가 실제로 0
# 구매결정o라고 예측한 데이터의 83%가 실제로 1
# 실제 구매결정x인 데이터의 90%가 0
# 실제 구매결정o인 데이터의 86%가 0

## ROC Curve

In [ ]:
from sklearn import metrics
def buildROC(test_target,test_preds):
    fpr, tpr, threshold = metrics.roc_curve(test_target,test_preds)
    roc_auc = metrics.auc(fpr, tpr)
    plt.title('Receiver Operating Characteristic')
    plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
    plt.legend(loc = 'lower right')
    plt.plot([0, 1], [0, 1],'r--')
    plt.ylabel('True Positive Rate')
    plt.xlabel('False Positive Rate')
    plt.gcf().savefig('roc.png')

In [ ]:
buildROC(test_target,xgb_pred)

### 결론: 그리드 서치 Xgboost 예측모델이 그리드서치 안했을 때보다 성능이 더 우수 (0.89->0.91)

### 모델 저장 및 불러오기(xgboost 내장함수 이용)

In [ ]:
# 파일명
filename = 'xgb_model.model'

In [ ]:
# 모델 저장
model.save_model(filename)

In [ ]:
# 모델 불러오기
new_xgb_model = xgb.XGBClassifier() # 모델 초기화
new_xgb_model.load_model(filename) # 모델 불러오기

### 이후 과정 -> 예측템플릿 작성,  Submission 파일 작업 및 내보내기